In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from autoemulate.compare import AutoEmulate
from sklearn.metrics import r2_score

from autoemulate.experimental_design import LatinHypercube
from autoemulate.simulations.flow_functions import FlowProblem

import json

show_progress = False if os.getenv("JUPYTER_BOOK_BUILD", "false").lower() == "true" else True

In [ ]:
# Load the simulation input parameters and summary statistics

n_samples = 32768 ## compare with separate notebook 32768

sim_path = f'./outputs/simulations/output_{n_samples}_samples/'

# Load the simulation input parameters
input_params = pd.read_csv(os.path.join(sim_path, f'input_samples_{n_samples}.csv'))

# Load the boolean mask for failed simulations
with open(os.path.join(sim_path, 'bool_indices.json'), 'r') as f:
    bool_indices = json.load(f)

# Load the summary statistics
summary_stats = pd.read_csv(os.path.join(sim_path, f'simulations_summary.csv'))

# Create folder for r2
save_folder = "outputs/emulator_r2/r2_values"
os.makedirs(save_folder, exist_ok=True)

r2 = eval_df['r2'].iloc[0]
r2_path = os.path.join(save_folder, f"r2_{n_samples}.txt")
with open(r2_path, "w") as f:
    f.write(str(r2))
print(f"Saved R² value ({r2:.4f}) to {r2_path}")

In [ ]:
# Get column names for parameters up to 'T'
parameter_names = list(input_params.columns[:input_params.columns.get_loc('T') + 1])

input_params[parameter_names]

In [ ]:
# Drop all the rows from input_params that correspond to entries in bool_indices
input_params = input_params.drop(bool_indices)

In [ ]:
Y = summary_stats['p_ao_mean'].values

In [ ]:
em = AutoEmulate()
em.setup(X = input_params[parameter_names], 
         y = Y, 
         models = ['gp'], ## Stick to gp
         n_jobs=-8)

In [ ]:
best_model = em.compare()

In [ ]:
em.summarise_cv()

In [ ]:
gp = em.get_model("GaussianProcess")
em.evaluate(gp)
eval_df = em.evaluate(gp)

In [ ]:
import os
save_folder = "outputs/emulator_r2/r2_values"
os.makedirs(save_folder, exist_ok=True)

r2_path = os.path.join(save_folder, f"r2_{n_samples}.txt")
with open(r2_path, "w") as f:
    f.write(str(r2))
print(f"Saved R² value ({r2:.4f}) to {r2_path}")

In [ ]:
em.plot_eval(gp)

In [ ]:
best_emulator = em.refit(gp)